In [ ]:
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import polyline
import geopandas as gpd
import geojson

load_dotenv()
api_key = os.getenv("google_api")
gmaps = googlemaps.Client(key=api_key)
GDRIVE_FOLDER = Path(os.getenv("GDRIVE_FOLDER"))
origins = GDRIVE_FOLDER / "origin.csv"
destinations = GDRIVE_FOLDER / "attractions.csv"
origin_df = pd.read_csv(origins)
destinations_df = pd.read_csv(destinations)
now = datetime.now()



origin = (
    39.946196,
    -75.139832,
)  # hardcoded here since my origin doesn't change, but if yours does you can tweak the function to accept origins.


def distance_duration_iteration():
    '''returns a list of origin/destination dicts for driving or transit, respectively'''
    driving_list = []
    transit_list = []

    def distance_duration(destination):
        output = gmaps.directions(
            origin,
            destination,
            mode,
            units="imperial",
            departure_time=now,
        )

        if mode == "driving":
            driving_list.append(output)

        if mode == "transit":
            transit_list.append(output)

    """loops through the destinations file and runs the distance duration function"""
    for idx, row in destinations_df.iterrows():
        lat = row["Latitude"]
        lon = row["Longitude"]
        destination = (lat, lon)
        distance_duration(destination)

    if mode == "driving":
        print("returning driving list...")
        return driving_list
    if mode == "transit":
        print("returning transit list...")
        return transit_list


def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the google api returns"""
    labels = ["distance", "duration", "lat", "lng"]
    df = pd.DataFrame(columns=labels)

    try:
        for item in list_of_dicts:
            destination_lat = item[0]["legs"][0]["end_location"]["lat"]
            destination_lng = item[0]["legs"][0]["end_location"]["lng"]
            distance = item[0]["legs"][0]["distance"]["text"]
            duration = item[0]["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)
    except:
        for item in list_of_dicts[0]:
            destination_lat = item["legs"][0]["end_location"]["lat"]
            destination_lng = item["legs"][0]["end_location"]["lng"]
            distance = item["legs"][0]["distance"]["text"]
            duration = item["legs"][0]["duration"]["text"]
            d = {
                "distance": [distance],
                "duration": [duration],
                "lat": [destination_lat],
                "lng": [destination_lng],
            }
            list_df = pd.DataFrame(data=d)
            df = pd.concat([list_df, df], axis=0, ignore_index=True)

    return df


def df_to_csv(df):
    df.to_csv(GDRIVE_FOLDER / f"{mode}_travel_times.csv", sep=",")

In [ ]:
mode = "driving"
driving_stored_in_memory = distance_duration_iteration()
mode = "transit"
transit_stored_in_memory = distance_duration_iteration()

In [70]:
mode = "transit"
from shapely import wkt

def unpack_geometries(transpo_list):
    '''accepts either the transit list or the driving list from the distance_duration_iteration function and parses accordingly'''
    transit_polylines = []
    driving_polylines = []
    item_count = 0
    if mode == "transit":
        for item in transpo_list:
            item_count += 1
            if not item:
                transit_polylines.append(item) #just appending blank items here if there's no polyline
            else:
                line = geojson.MultiLineString(polyline.decode(item[0]["overview_polyline"]["points"], geojson = True))
                 #appending polylines

        df = pd.DataFrame(line)           
    if mode == "driving":
        print("this is a driving list")
    
    return df.head(800)

def unpack_geometries_playground(transpo_list):
    '''accepts either the transit list or the driving list from the distance_duration_iteration function and parses accordingly'''
    if mode == "transit":
        polylines = []
        for item in transpo_list:
            if not item:
                pass #just appending blank items here if there's no polyline
            else:
                line = geojson.MultiLineString(polyline.decode(item[0]["overview_polyline"]["points"], geojson = True))
                polylines.append(line)
        df = pd.DataFrame(polylines[0])
        df["geometry"] = df['coordinates'].apply(wkt.loads)


        gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="coordinates")
    # gdf.set_geometry("coordinates", drop=False, inplace=False, crs=None)
    return gdf

    
    

# unpack_geometries(transit_stored_in_memory)
unpack_geometries_playground(transit_stored_in_memory)
# polylines_to_gdf(unpack_geometries(transit_stored_in_memory))

TypeError: Only str is accepted.